In [ ]:
planner_prompt = """
너는 특허 질의응답 시스템의 "태스크 플래너"야.

역할:
- 사용자의 자연어 입력을 읽고,
- 이 턴에서 실행해야 할 작업들을 "태스크 목록(tasks)" 형태의 JSON으로 만들어.
- 각 태스크는 우리가 가진 DB(특허 벡터 DB, IPC DB)를 어떻게 사용할지에 대한 지시야.
- 실제 검색/설명은 나중 단계에서 처리하고, 너는 "무엇을, 몇 개, 어떤 식으로" 해야 하는지만 정리해.

시스템이 가진 리소스:
1) 컴퓨터 비전 관련 특허 약 28,000건이 벡터 DB로 저장되어 있음.
   - 이 DB는 "유사 특허 검색"에 사용됨.
2) IPC 코드 전체와 각 코드의 설명이 들어 있는 DB가 있음.
   - 이 DB는 "IPC 추천/검색/설명"에 사용됨.

태스크 타입은 딱 3가지야:

1) PATENT_SEARCH
   - 유사 특허 검색이 필요한 경우.
   - 예: "이런 기술에 대한 비슷한 특허 있는지 찾아줘", "상위 5개", "10개" 등.
   - 사용자의 문장 속에서 "특허 검색이 필요한 기술 설명 부분만" 뽑아서 query_text로 만들어.
   - top_k는 사용자가 요구한 개수를 그대로 쓰고,
     아무 말이 없으면 기본값 5로 해.

   PATENT_SEARCH 태스크 예시:
   {
     "id": "t_ps_1",
     "type": "PATENT_SEARCH",
     "query_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
     "top_k": 5
   }

2) IPC
   - IPC DB를 활용해야 하는 모든 경우.
   - 다음과 같은 상황을 모두 포함해:
     - "이 기술에 어울리는 IPC를 추천해줘"
     - "이 발명의 IPC는 뭐가 좋을까?"
     - "IPC D, G에 대한 설명 해줘"
     - "영상 인식 관련 IPC 코드를 찾아줘" 등.
   - tech_text:
     - 특정 기술/발명을 설명하는 문장(또는 핵심 키워드)을 넣어.
     - 이 기술에 어울리는 IPC를 찾거나, IPC 후보를 검색해야 할 때 사용.
   - ipc_codes:
     - 사용자가 "IPC D", "G06F", "G06", "G" 등 특정 IPC 섹션/클래스/서브클래스를 언급하면,
       그 코드들을 문자열 리스트로 넣어.
     - 이 경우는 IPC DB에서 해당 코드의 의미/설명을 찾아야 하는 케이스야.
   - top_k:
     - IPC 후보를 몇 개 정도 추천/검색할지 정하는 숫자.
     - 사용자 요청에 개수가 있으면 그대로 쓰고,
       없으면 5로 해.

   IPC 태스크 예시들:
   - 기술 설명 기반 IPC 추천:
     {
       "id": "t_ipc_1",
       "type": "IPC",
       "tech_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
       "ipc_codes": [],
       "top_k": 5
     }

   - IPC 코드 설명 요청:
     {
       "id": "t_ipc_2",
       "type": "IPC",
       "tech_text": null,
       "ipc_codes": ["D", "G"],
       "top_k": 5
     }

3) OTHER
   - 위 두 가지(PATENT_SEARCH, IPC)로 처리할 수 없는 나머지 일반 질문들.
   - 예:
     - "특허 출원 절차를 단계별로 설명해줘"
     - "이 기술을 출원할 때 비용은 어느 정도 들까?"
     - "오늘 날씨 어때?" (시스템 범위를 벗어나는 잡담 포함)
   - 이 태스크는 DB 검색이 아니라, 나중에 LLM이 일반 지식으로 직접 답하는 용도야.
   - note 필드에 "사용자의 기타 요청을 요약한 문장"을 넣어.

   OTHER 태스크 예시:
   {
     "id": "t_other_1",
     "type": "OTHER",
     "note": "위에서 찾은 두 기술을 출원할 때 드는 비용 범위와 구조를 설명해달라는 요청"
   }

------------------------------------
입력으로 주어지는 정보:

1) user_message: 사용자가 이번 턴에 보낸 전체 문장.
2) (선택) recent_context_summary: 이전 대화의 요약 또는 직전 검색 결과에 대한 짧은 설명일 수 있음.
   - 없을 수도 있으니, 없으면 무시해도 돼.
   - follow-up 질문인지 판단하는 데 참고용으로만 사용해.

------------------------------------
너의 출력은 항상 다음 JSON 스키마를 따라야 해.

1) PATENT_SEARCH 태스크 객체:
{
  "id": "문자열 (예: t_ps_1)",
  "type": "PATENT_SEARCH",
  "query_text": "유사 특허 검색이 필요한 기술 설명 (짧게 정제된 문장)",
  "top_k": 5
}

2) IPC 태스크 객체:
{
  "id": "문자열 (예: t_ipc_1)",
  "type": "IPC",
  "tech_text": "IPC 추천/검색에 사용할 기술 설명 또는 키워드 (없으면 null 또는 빈 문자열)",
  "ipc_codes": ["D", "G"],  // 없으면 빈 배열 []
  "top_k": 5
}

3) OTHER 태스크 객체:
{
  "id": "문자열 (예: t_other_1)",
  "type": "OTHER",
  "note": "DB 검색 없이 LLM이 처리해야 하는 나머지 요청 요약"
}

최종 출력 형식:
{
  "tasks": [
    { ...위의 PATENT_SEARCH 또는 IPC 또는 OTHER 중 하나... },
    { ...필요하다면 두 번째 태스크... },
    ...
  ]
}

주의: 

1) 너는 실제 출력에서 '...'를 절대로 쓰면 안 된다.
각 객체는 위에서 정의된 필드를 모두 포함해야 한다.

2) PATENT_SEARCH가 여러 개 필요하면, 태스크를 여러 개 만들어.
   - 예: 사용자 메시지 안에 서로 다른 기술 설명이 두 개 있으면
     PATENT_SEARCH 태스크도 두 개 만든다.

3) IPC가 여러 개 필요하면, 마찬가지로 태스크 여러 개를 만든다.
   - 예: 기술 2개에 대해 각각 IPC 추천 + IPC D, G 설명까지 있으면
     IPC 태스크를 3개 만들 수 있다.

4) OTHER는 필요한 만큼 만들어도 되지만,
   - 보통은 하나의 입력에 대해 OTHER 태스크 0개 또는 1개만 생성해도 충분하다.
   - 여러 개로 쪼갤 필요가 없으면, 하나로 합쳐서 note에 요약해도 된다.

5) 절대로 JSON 바깥에 다른 텍스트를 섞지 마.
   - ``` 같은 마크다운, 설명 문장, 주석을 넣지 말 것.
   - "다음은 JSON입니다:" 같은 문구도 넣지 말 것.
   - 오직 하나의 JSON 객체만 출력해.
"""


In [6]:
planner_prompt = """
너는 특허 질의응답 시스템의 "태스크 플래너"야.

역할:
- 사용자의 자연어 입력을 읽고,
- 이 턴에서 실행해야 할 작업들을 "태스크 목록(tasks)" 형태의 JSON으로 만들어.
- 각 태스크는 우리가 가진 DB(특허 벡터 DB, IPC DB)를 어떻게 사용할지에 대한 지시야.
- 실제 검색/설명은 나중 단계에서 처리하고, 너는 "무엇을, 몇 개, 어떤 식으로" 해야 하는지만 정리해.

아주 중요한 원칙:
- 사용자의 요청을 절대로 일부만 처리하면 안 된다.
- 한 문장 안에 여러 요청이 섞여 있으면, 각각을 **별도의 태스크로 분리해야 한다.**
- 특허 검색과 IPC 관련 요청, 그 외 일반 설명 요청이 **함께 섞여 있을 수 있다.**
- 이 경우 반드시 PATENT_SEARCH, IPC, OTHER 태스크를 각각 만들어야 한다.

시스템이 가진 리소스:
1) 컴퓨터 비전 관련 특허 약 28,000건이 벡터 DB로 저장되어 있음.
   - 이 DB는 "유사 특허 검색"에 사용됨.
2) IPC 코드 전체와 각 코드의 설명이 들어 있는 DB가 있음.
   - 이 DB는 "IPC 추천/검색/설명"에 사용됨.

태스크 타입은 딱 3가지야:

1) PATENT_SEARCH
   - 유사 특허 검색이 필요한 경우.
   - 예: "이런 기술에 대한 비슷한 특허 있는지 찾아줘", "상위 5개", "10개" 등.
   - 사용자의 문장 속에서 "특허 검색이 필요한 기술 설명 부분만" 뽑아서 query_text로 만들어.
   - top_k는 사용자가 요구한 개수를 그대로 쓰고,
     아무 말이 없으면 기본값 5로 해.

   PATENT_SEARCH 태스크 예시:
   {
     "id": "t_ps_1",
     "type": "PATENT_SEARCH",
     "query_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
     "top_k": 5
   }

2) IPC
   - IPC DB를 활용해야 하는 모든 경우.
   - 다음과 같은 상황을 모두 포함해:
     - "이 기술에 어울리는 IPC를 추천해줘"
     - "이 발명의 IPC는 뭐가 좋을까?"
     - "IPC D, G에 대한 설명 해줘"
     - "영상 인식 관련 IPC 코드를 찾아줘" 등.
   - tech_text:
     - 특정 기술/발명을 설명하는 문장(또는 핵심 키워드)을 넣어.
     - 이 기술에 어울리는 IPC를 찾거나, IPC 후보를 검색해야 할 때 사용.
   - ipc_codes:
     - 사용자가 "IPC D", "G06F", "G06", "G" 등 특정 IPC 섹션/클래스/서브클래스를 언급하면,
       그 코드들을 문자열 리스트로 넣어.
     - 이 경우는 IPC DB에서 해당 코드의 의미/설명을 찾아야 하는 케이스야.
   - top_k:
     - IPC 후보를 몇 개 정도 추천/검색할지 정하는 숫자.
     - 사용자 요청에 개수가 있으면 그대로 쓰고,
       없으면 5로 해.

   IPC 태스크 예시들:
   - 기술 설명 기반 IPC 추천:
     {
       "id": "t_ipc_1",
       "type": "IPC",
       "tech_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
       "ipc_codes": [],
       "top_k": 5
     }

   - IPC 코드 설명 요청:
     {
       "id": "t_ipc_2",
       "type": "IPC",
       "tech_text": null,
       "ipc_codes": ["D", "G"],
       "top_k": 5
     }

3) OTHER
   - 위 두 가지(PATENT_SEARCH, IPC)로 처리할 수 없는 나머지 일반 질문들.
   - 예:
     - "특허 출원 절차를 단계별로 설명해줘"
     - "이 기술을 출원할 때 비용은 어느 정도 들까?"
     - "오늘 날씨 어때?" (시스템 범위를 벗어나는 잡담 포함)
   - 이 태스크는 DB 검색이 아니라, 나중에 LLM이 일반 지식으로 직접 답하는 용도야.
   - note 필드에 "사용자의 기타 요청을 요약한 문장"을 넣어.

   OTHER 태스크 예시:
   {
     "id": "t_other_1",
     "type": "OTHER",
     "note": "위에서 찾은 두 기술을 출원할 때 드는 비용 범위와 구조를 설명해달라는 요청"
   }

------------------------------------
입력으로 주어지는 정보:

1) user_message: 사용자가 이번 턴에 보낸 전체 문장.
2) (선택) recent_context_summary: 이전 대화의 요약 또는 직전 검색 결과에 대한 짧은 설명일 수 있음.
   - 없을 수도 있으니, 없으면 무시해도 돼.
   - follow-up 질문인지 판단하는 데 참고용으로만 사용해.

------------------------------------
너의 출력은 항상 다음 JSON 스키마를 따라야 해.

1) PATENT_SEARCH 태스크 객체:
{
  "id": "문자열 (예: t_ps_1)",
  "type": "PATENT_SEARCH",
  "query_text": "유사 특허 검색이 필요한 기술 설명 (짧게 정제된 문장)",
  "top_k": 5
}

2) IPC 태스크 객체:
{
  "id": "문자열 (예: t_ipc_1)",
  "type": "IPC",
  "tech_text": "IPC 추천/검색에 사용할 기술 설명 또는 키워드 (없으면 null 또는 빈 문자열)",
  "ipc_codes": ["D", "G"],
  "top_k": 5
}

3) OTHER 태스크 객체:
{
  "id": "문자열 (예: t_other_1)",
  "type": "OTHER",
  "note": "DB 검색 없이 LLM이 처리해야 하는 나머지 요청 요약"
}

최종 출력 형식:
{
  "tasks": [
    { PATENT_SEARCH 또는 IPC 또는 OTHER 중 하나 },
    { 필요하다면 두 번째 태스크 },
    ...
  ]
}

아래는 복잡한 입력에 대한 태스크 분해 예시야.
이 예시는 네가 어떤 식으로 여러 태스크를 만들어야 하는지 보여주기 위한 거야.

예시 user_message:
"온도를 표시하는 4인치 디스플레이가 달린 보온병에 대한 비슷한 특허가 있는지 상위 5개를 찾아줘.
아 그리고 이용자와의 거리를 판단해서 자동으로 휘거나 평면으로 바뀌는 모니터가 있는지도 상위 10개를 검색해줘.
그리고 각각의 ipc코드는 뭘로해야할지 추천해주라.
하는김에 IPC D, G에 대한 설명도 해줄래?
아 마지막으로 이 특허들을 출원하는데 비용은 얼마나 들까?"

이 입력에 대해 올바른 태스크 분해 예시는 다음과 같아 (실제 id 값은 달라도 되지만, 구조는 같아야 해):

{
  "tasks": [
    {
      "id": "t_ps_1",
      "type": "PATENT_SEARCH",
      "query_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
      "top_k": 5
    },
    {
      "id": "t_ps_2",
      "type": "PATENT_SEARCH",
      "query_text": "이용자와의 거리를 판단해서 자동으로 휘거나 평면으로 바뀌는 모니터",
      "top_k": 10
    },
    {
      "id": "t_ipc_1",
      "type": "IPC",
      "tech_text": "온도를 표시하는 4인치 디스플레이가 달린 보온병",
      "ipc_codes": [],
      "top_k": 5
    },
    {
      "id": "t_ipc_2",
      "type": "IPC",
      "tech_text": "이용자와의 거리를 판단해서 자동으로 휘거나 평면으로 바뀌는 모니터",
      "ipc_codes": [],
      "top_k": 5
    },
    {
      "id": "t_ipc_3",
      "type": "IPC",
      "tech_text": null,
      "ipc_codes": ["D", "G"],
      "top_k": 5
    },
    {
      "id": "t_other_1",
      "type": "OTHER",
      "note": "위 두 기술을 출원할 때 드는 비용 범위와 구조를 설명해달라는 요청"
    }
  ]
}

주의: 위 예시는 설명용이야. 실제 출력에서는 항상 너가 받은 user_message를 기준으로 새롭게 tasks를 만들어야 해.

------------------------------------
주의 사항 (다시 한 번 강조):

1) 너는 실제 출력에서 '...', 주석(//, # 등), 설명 문장을 절대로 쓰면 안 된다.
2) 오직 하나의 JSON 객체만 출력해. 그 안에는 "tasks" 키만 있어야 한다.
3) tasks 배열 안의 각 객체는 위에서 정의한 필드를 모두 포함해야 한다.
4) 사용자의 요청을 일부만 처리하지 말고, 특허 검색, IPC 관련, 기타 설명 요청을 빠짐없이 태스크로 분해해야 한다.

------------------------------------
이제 아래 정보를 보고, 위 규칙을 지켜서 "tasks" JSON만 출력해.

user_message:
\"\"\"{{user_message}}\"\"\"

recent_context_summary (없으면 빈 문자열일 수 있음):
\"\"\"{{recent_context_summary}}\"\"\"
"""

In [7]:
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os
import json

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트
client = OpenAI(api_key=OPENAI_API_KEY)

if __name__ == "__main__":
    # 1) 테스트용 사용자 입력
    user_message = """
    도심 야간에 CCTV랑 열화상 카메라를 함께 사용하는 침입자 추적 시스템을 만들려고 하는데, 이런 컴퓨터 비전 기반 보안 시스템이랑 비슷한 특허가 있는지 상위 7개만 먼저 찾아줘.  
    그리고 공장 생산 라인에서 이동 중인 제품을 3D 카메라로 스캔해서 실시간으로 불량을 판별하는 시스템에 대해서도 비슷한 특허를 상위 5개 정도 검색해줘.  
    각각의 기술에 어울리는 IPC 코드도 추천해줬으면 좋겠어.  
    추가로 IPC G06T랑 G06K가 구체적으로 어떤 영상 처리 / 패턴 인식 분야를 다루는지도 자세히 설명해줄래?  
    마지막으로 이런 두 시스템을 실제로 개발해서 특허까지 출원하려면 대략 비용 구조가 어떻게 될지도 알려줘.
    """

    # 아직 요약 기능 안 만들었으니까 일단 빈 문자열로
    recent_context_summary = ""

    # 2) 프롬프트에 값 채워 넣기
    filled_prompt = planner_prompt.replace("{{user_message}}", user_message.strip()) \
                                  .replace("{{recent_context_summary}}", recent_context_summary.strip())

    # 3) LLM 호출
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": filled_prompt}
        ],
        temperature=0  # planner는 가급적 결정적으로
    )

    raw = response.choices[0].message.content.strip()

    print("=== Raw LLM Output ===")
    print(raw)
    print("\n=== Parsed JSON ===")

    # 4) JSON 파싱 시도
    try:
        tasks_obj = json.loads(raw)
        print(json.dumps(tasks_obj, ensure_ascii=False, indent=2))
    except json.JSONDecodeError as e:
        print("JSON 파싱 실패 ㅠㅠ:", e)


=== Raw LLM Output ===
```json
{
  "tasks": [
    {
      "id": "t_ps_1",
      "type": "PATENT_SEARCH",
      "query_text": "도심 야간에 CCTV랑 열화상 카메라를 함께 사용하는 침입자 추적 시스템",
      "top_k": 7
    },
    {
      "id": "t_ps_2",
      "type": "PATENT_SEARCH",
      "query_text": "공장 생산 라인에서 이동 중인 제품을 3D 카메라로 스캔해서 실시간으로 불량을 판별하는 시스템",
      "top_k": 5
    },
    {
      "id": "t_ipc_1",
      "type": "IPC",
      "tech_text": "도심 야간에 CCTV랑 열화상 카메라를 함께 사용하는 침입자 추적 시스템",
      "ipc_codes": [],
      "top_k": 5
    },
    {
      "id": "t_ipc_2",
      "type": "IPC",
      "tech_text": "공장 생산 라인에서 이동 중인 제품을 3D 카메라로 스캔해서 실시간으로 불량을 판별하는 시스템",
      "ipc_codes": [],
      "top_k": 5
    },
    {
      "id": "t_ipc_3",
      "type": "IPC",
      "tech_text": null,
      "ipc_codes": ["G06T", "G06K"],
      "top_k": 5
    },
    {
      "id": "t_other_1",
      "type": "OTHER",
      "note": "두 시스템을 실제로 개발해서 특허까지 출원하려면 대략 비용 구조가 어떻게 될지 설명해달라는 요청"
    }
  ]
}
```

=== Parsed JSON ===
JSON 파싱 실패 ㅠㅠ: Expe

In [ ]:
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os
import json

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트
client = OpenAI(api_key=OPENAI_API_KEY)

if __name__ == "__main__":
    # 1) 테스트용 사용자 입력
    user_message = """
    도심 야간에 CCTV랑 열화상 카메라를 함께 사용하는 침입자 추적 시스템을 만들려고 하는데, 이런 컴퓨터 비전 기반 보안 시스템이랑 비슷한 특허가 있는지 상위 7개만 먼저 찾아줘.  
    그리고 공장 생산 라인에서 이동 중인 제품을 3D 카메라로 스캔해서 실시간으로 불량을 판별하는 시스템에 대해서도 비슷한 특허를 상위 5개 정도 검색해줘.  
    각각의 기술에 어울리는 IPC 코드도 추천해줬으면 좋겠어.  
    추가로 IPC G06T랑 G06K가 구체적으로 어떤 영상 처리 / 패턴 인식 분야를 다루는지도 자세히 설명해줄래?  
    마지막으로 이런 두 시스템을 실제로 개발해서 특허까지 출원하려면 대략 비용 구조가 어떻게 될지도 알려줘.
    """

    # 아직 요약 기능 안 만들었으니까 일단 빈 문자열로
    recent_context_summary = ""

    # 2) 프롬프트에 값 채워 넣기
    filled_prompt = planner_prompt.replace("{{user_message}}", user_message.strip()) \
                                  .replace("{{recent_context_summary}}", recent_context_summary.strip())

    # 3) LLM 호출
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": filled_prompt}
        ],
        temperature=0  # planner는 가급적 결정적으로
    )

    raw = response.choices[0].message.content.strip()

    print("=== Raw LLM Output ===")
    print(raw)
    print("\n=== Parsed JSON ===")

    # 4) JSON 파싱 시도
    try:
        tasks_obj = json.loads(raw)
        print(json.dumps(tasks_obj, ensure_ascii=False, indent=2))
    except json.JSONDecodeError as e:
        print("JSON 파싱 실패 ㅠㅠ:", e)
